<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%237.keras%20Tuner%EB%A1%9C%20%EC%B4%88%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98%20%EB%AF%B8%EC%84%B8%20%EC%A1%B0%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

    이미지 분류 애플리케이션에 하이퍼튜닝 수행


In [2]:
import tensorflow as tf
from tensorflow import keras
import IPython

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 96 kB 2.3 MB/s 


In [4]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [5]:
# Dataset
(img_train, label_train),(img_test, label_test) = keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
# Model Builder
# - 컴파일된 모델 반환 -> 인라인으로 정의한 하이퍼파라미터 사용하여 모델 하이퍼튜닝
def model_builder(hp) :
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape = (28,28)))

    # Tune the number of units in the 1st Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for optimizer
    # Choose an optimal value between 0.01, 0.001, 0.0001
    hp_lr = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_lr),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [10]:
# 튜너를 인스턴스화 후 하이퍼 튜닝 진행
# 케라스 튜너에는 4가지 종류 있음
# RandomSearch, Hyperband, BayesianOptimization, Sklearn

# Hyperband 튜너 인스턴스화
# - 최적화할 하이퍼모델인 objective, 및 훈련할 최대 epoch 수 지정
# - 1+log_factor(max_epochs) 계산 -> 가장 가까운 정수로 반환하여 훈련할 모델 수 결정

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [11]:
# 하이퍼 파라미터 검색을 실행하기 전 훈련 단계가 끝날 때 마다 훈련 결과를 지우도록 콜백 정의

class ClearTrainingOutput(tf.keras.callbacks.Callback) :
    def on_train_end(*args, **kwargs) :
        IPython.display.clear_output(wait = True)

In [12]:
# 하이퍼 파라미터 검색 실행 
tuner.search(img_train, label_train,
             epochs= 10,
             validation_data = (img_test, label_test),
             callbacks = [ClearTrainingOutput()])

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8881999850273132

Best val_accuracy So Far: 0.8881999850273132
Total elapsed time: 00h 16m 21s
INFO:tensorflow:Oracle triggered exit


In [13]:
# Get the best parameters 

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [14]:
# 최적의 파라미터로 재훈련

model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train,
          validation_data = (img_test, label_test),
          epochs = 10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4837 - accuracy: 0.8292 - val_loss: 0.4198 - val_accuracy: 0.8461
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3642 - accuracy: 0.8672 - val_loss: 0.4111 - val_accuracy: 0.8548
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3272 - accuracy: 0.8791 - val_loss: 0.3855 - val_accuracy: 0.8576
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3050 - accuracy: 0.8862 - val_loss: 0.3435 - val_accuracy: 0.8757
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2861 - accuracy: 0.8943 - val_loss: 0.3515 - val_accuracy: 0.8709
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2694 - accuracy: 0.8995 - val_loss: 0.3452 - val_accuracy: 0.8775
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2591 - accuracy: 0.9029 - val_loss: 0.3423 - val_accuracy: